In [ ]:
 import pandas as pd 

In [ ]:
# as we keep pulling sample from the existing datasheet, it is critical to exlcude them in further pulling
df = pd.read_csv("filtered_sequenceable_sample.csv")
df2 = pd.read_csv("2024_first_pulling.csv")

# Identify MPI values in the first pulling file
mpi_values_to_exclude = set(df2['MPI'].unique())

# Drop rows from the meta_with_fips_df where MPI matches those in mpi_values_to_exclude
filtered_meta_with_fips_df = df[~df['MPI'].isin(mpi_values_to_exclude)]

filtered_meta_with_fips_df.to_csv('03_27_metadata_sequenceable.csv', index=False)
# this dataframe is the sequencable sample that we should focus in the future

In [ ]:
#next I would like to assign 12 digits fips code based on the metdata 

data = pd.read_csv('previsouly_generate_file.csv')
data['CensusBlock'] = data['CensusBlock'].astype(str).replace(r'\.0$', '', regex=True)
data['block_group'] = data['CensusBlock'].astype(str).str[0]

In [ ]:
data['CountyFIPS'] = data['CountyFIPS'].astype(str).replace(r'\.0$', '', regex=True)
data['CensusTract'] = data['CensusTract'].astype(str).replace(r'\.0$', '', regex=True).str.pad(6,fillchar='0')

data['FIPS_temp'] = data.CountyFIPS.str.cat(data.CensusTract)

data['FIPS_temp'] = data['FIPS_temp'].astype(str).replace(r'\.0$', '', regex=True)
data['CensusBlock'] = data['CensusBlock'].astype(str).replace(r'\.0$', '', regex=True)
data['block_group'] = data['CensusBlock'].astype(str).str[0]

data['FIPS'] = data.FIPS_temp.str.cat(data.block_group)

data = data.drop('FIPS_temp', axis=1)
#save the data with complete 12 digit FIPS code
data.to_csv('freezer_available_04_22_with_fips_12digit.csv')

In [ ]:
#assign ADI based on the FIPS code 
data = pd.read_csv(''freezer_available_04_22_with_fips_12digit.csv')
adi = pd.read_csv('WI_2021_ADI_Census_Block_Group_v4_0_1.csv')
data['CensusBlock'] = data['CensusBlock'].astype(str).replace(r'\.0$', '', regex=True)
data['block_group'] = data['CensusBlock'].astype(str).str[0]
data['CountyFIPS'] = data['CountyFIPS'].astype(str).replace(r'\.0$', '', regex=True)
data['CensusTract'] = data['CensusTract'].astype(str).replace(r'\.0$', '', regex=True).str.pad(6,fillchar='0')

data['FIPS_temp'] = data.CountyFIPS.str.cat(data.CensusTract)

data['FIPS_temp'] = data['FIPS_temp'].astype(str).replace(r'\.0$', '', regex=True)
data['CensusBlock'] = data['CensusBlock'].astype(str).replace(r'\.0$', '', regex=True)
data['block_group'] = data['CensusBlock'].astype(str).str[0]

data['FIPS'] = data.FIPS_temp.str.cat(data.block_group)
data = data.drop('FIPS_temp', axis=1)
adi['FIPS'] = adi['FIPS'].astype(str).replace(r'\.0$', '', regex=True)
data['FIPS'] = data['FIPS'].astype(str).replace(r'\.0$', '', regex=True)


data_w_adi = pd.merge(data, adi, how='left', on=['FIPS'])
data_w_adi.to_csv('freezer_available_04_22_with_ADI.csv')



In [ ]:

#assign RUCA code based on the FIPS code
meta_with_fips_df = pd.read_csv('freezer_available_04_22_with_fips_12digit.csv')
ruca_data_df = pd.read_csv('RUCA-Definitions.csv')

# Ensure both FIPS and GEOCODE are treated as strings
meta_with_fips_df['FIPS'] = meta_with_fips_df['FIPS'].astype(str)
# Ensure the correct column for GEOCODE (Total-FIPS-Code) is used
ruca_data_df['Total-FIPS-Code'] = ruca_data_df['Total-FIPS-Code'].astype(str)

# Truncate FIPS to the first 11 characters to match with Total-FIPS-Code
meta_with_fips_df['FIPS_RUCA'] = meta_with_fips_df['FIPS'].str[:11]

# Merge the dataframes on the prepared FIPS and GEOCODE columns
merged_df = pd.merge(meta_with_fips_df, ruca_data_df, left_on='FIPS_RUCA', right_on='Total-FIPS-Code', how='left')

# Check for missing information post-merge
missing_info = merged_df[merged_df['Total-FIPS-Code'].isna()]
print(f"Rows with missing RUCA info: {missing_info.shape[0]}")

missing_info = merged_df[merged_df['ADI_STATERNK'].isna()]
print(f"Rows with missing ADI info: {missing_info.shape[0]}")

merged_df.dropna(subset=['State-County FIPS Code'], inplace=True)
#save the merged dataframe
merged_df.to_csv("freezer_available_04_22_with_fips_12digiit_RUCA.csv")